In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lbgm
#from xgboost import XGBRegressor
#import xgboost as xgb
from catboost import CatBoostRegressor
#from lightgbm.sklearn import lbgmRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [2]:
sys.path.append(os.path.realpath('..'))

In [3]:
train_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\train.csv")
test_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\test.csv")
sample_sub = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\sample_submission.csv")

In [4]:
columns_to_vectorize = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for vector_target in columns_to_vectorize:
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

In [5]:
features = ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_v', 'ever_married_v', 'work_type_v', 'Residence_type_v', 'smoking_status_v']
target = ['stroke']

In [6]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [7]:
def cat_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'depth': trial.suggest_int('depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        #'rsm': trial.suggest_float('rsm', 0.001, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), 
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 20),
        'random_strength': trial.suggest_float('random_strength', 0.001, 0.9),
    } 

    clf = CatBoostRegressor(iterations=20000,
                            **param_grid,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            #loss_function='Logloss',
                            eval_metric='AUC',
                            task_type="GPU",
                            random_state=1,)
    
    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
    preds = clf.predict(X_valid)

    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    cat_ranks[roc_auc] = clf

    return roc_auc

In [8]:
cat_ranks = {}

train_time = 1 * 60 * 5
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='CATRegressor')
study.optimize(cat_objective, timeout=train_time)



[I 2023-01-16 08:57:09,505] A new study created in memory with name: CATRegressor
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8879488	best: 0.8879488 (0)	total: 25.6ms	remaining: 8m 31s
bestTest = 0.8999465942
bestIteration = 148
Shrink model to first 149 iterations.
0.8999465941838171


[I 2023-01-16 08:57:16,744] Trial 0 finished with value: 0.8999465941838171 and parameters: {'depth': 5, 'learning_rate': 0.014854778598166467, 'subsample': 0.6004036544283574, 'min_data_in_leaf': 64, 'l2_leaf_reg': 6, 'random_strength': 0.5462398609623398}. Best is trial 0 with value: 0.8999465941838171.
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7692027	best: 0.7692027 (0)	total: 13.2ms	remaining: 4m 23s
bestTest = 0.8868779957
bestIteration = 47
Shrink model to first 48 iterations.
0.8868779956588413


[I 2023-01-16 08:57:20,612] Trial 1 finished with value: 0.8868779956588413 and parameters: {'depth': 2, 'learning_rate': 0.07137331001021455, 'subsample': 0.33249562648175224, 'min_data_in_leaf': 49, 'l2_leaf_reg': 3, 'random_strength': 0.7071469323405625}. Best is trial 0 with value: 0.8999465941838171.
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

bestTest = 0.8880575705
bestIteration = 134
Shrink model to first 135 iterations.
0.8880575705269935


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8480254	best: 0.8480254 (0)	total: 12.6ms	remaining: 4m 11s


[W 2023-01-16 09:05:04,095] Trial 77 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_27408\4211095094.py", line 24, in cat_objective
    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 2355, in _fit
    self._train(
  File "D:\source\repos\venv\Python310\lib\site-packages\catboost\core.py", line 1759, in _train
    self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._objec

KeyboardInterrupt: 

In [9]:
cat_ranks_bytes = pickle.dumps(cat_ranks)
with open("rendered_data/CAT_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(cat_ranks_bytes)

In [10]:
percent = math.ceil(len(cat_ranks.keys()) * .1)
top = sorted(list(cat_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(cat_ranks[key].predict(X))
    test_preds.append(cat_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 288.92it/s]


In [11]:
CAT_train_submission = pd.DataFrame(data={'id': train_df.id, 'stroke': train_final_preds})
CAT_train_submission.to_csv(fr'rendered_data/CAT_train_submission.csv', index=False)

CAT_test_submission = pd.DataFrame(data={'id': test_df.id, 'stroke': test_final_preds})
CAT_test_submission.to_csv(fr'rendered_data/CAT_test_submission.csv', index=False)